In [3]:
import sys
sys.path.append('/home/ubuntu/intelliscope/modules')

#!pip install neptune-client
import os
import neptune

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *
from neptune.sessions import Session
import custom_metrics


################################################################################################################
#project_name='Patientwise-Test-HYBRID-balanced'
project_name='Kunu'
prev_project_name='HYBRID-Imbalanced'
TrainTestVal='test'
thres=0.7
################################################################################################################


NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/'+prev_project_name]
experiments=project.get_experiments()
X_test,Y_test,names = instead_data_loaders(experiments[0].get_parameters(),TrainTestVal, True) # because we have separate projects for balanced, imbalanced datasets


def get_results(model=None, X_test_pcg=None, X_test_ecg=None, Y_test=None, threshold = thres):    
    y_pred = model.predict([X_test_pcg,X_test_ecg], batch_size=32, verbose=1)
    y_test = np.argmax(Y_test,axis=1)
    
    print(custom_metrics._class_metrics(y_test, y_pred[:, 1] > threshold))
    return y_test, y_pred


def get_individual_metrics(y_test, y_pred, names):
    patient_preds={}
    for i in range(len(names)): # create dict: patient_record_name: [start_point, label, [pred0, pred1]] 
        name=names[i]
        record_name,record_start_point,_=name.strip().split('_')
        if record_name in patient_preds:
            patient_preds[record_name].append([int(record_start_point),y_test[i], list(y_pred[i])])
        else:
            patient_preds[record_name]=[[int(record_start_point),y_test[i], list(y_pred[i])]]

    y_test_bulk, y_pred_bulk=[],[]    
    for key in patient_preds: # return :: patient-wise y_test, y_preds
        patient_pred=patient_preds[key]
        label, pred_label=get_single_patient_pred(patient_pred, mean)

        y_test_bulk.append(label)
        y_pred_bulk.append(pred_label)

    y_test_bulk=np.array(y_test_bulk)
    y_pred_bulk=np.array(y_pred_bulk)
    print(custom_metrics._class_metrics(y_test_bulk, y_pred_bulk[:, 1] > thres))
    
    return y_test_bulk, y_pred_bulk

def mean(list_):
    metric=np.mean(list_)
    return [1-metric,metric]

def get_single_patient_pred(patient_pred, func):
    #print(patient_pred)
    patient_pred.sort()
    
    y=np.array(patient_pred)[:,1]
    
    assert y.sum()%(y.shape[0])==0
    label=y[0] # label of the patient
    
    rows=[]
    for row in sorted(patient_pred):
        rows.append(list(row[2]))
    rows=np.array(rows)

    y_preds=np.argmax(rows, axis=1)
    pred_label=func(y_preds)
    return label, pred_label

def neptune_log_metrics(y_test=None, y_pred=None, threshold = thres):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)


Total  707
199 Images loaded across 2 Categories. Ignored blanks  60
508 Images loaded across 2 Categories. Ignored blanks  9
638 Total After removing blanks


In [4]:

X_test_pcg = X_test[:,:,:,0:3]/255.0
X_test_ecg = X_test[:,:,:,3:6]/255.0

for i in range(len(experiments)):
    if i==0:continue
    experiment    = experiments[i]
    model_name = str(experiment)+'.h5'
    PARAMS     = experiment.get_parameters()
    PARAMS['tags']=PARAMS['tags'][2:-2].strip().split("', '")
    
    model = load_model(PARAMS['modelsave_dir']+'/'+model_name)
    y_test, y_pred=get_results(model, X_test_pcg, X_test_ecg, Y_test)
    
    y_test_patientwise, y_pred_patientwise=get_individual_metrics(y_test, y_pred, names)

    
    neptune.init('intelliscope/'+project_name,NEPTUNE_API_TOKEN)
    exp = neptune.create_experiment(name=str(experiment),description=PARAMS['name'],params=PARAMS,tags=PARAMS['tags'],upload_stdout=True)
    neptune_log_metrics(y_test_patientwise, y_pred_patientwise)
    neptune.stop()

638/638 [==============================] - 3s 4ms/sample
{'accuracy': 0.7931034482758621, 'precision': 0.8895966029723992, 'recall': 0.8396793587174348, 'f1_score': 0.8639175257731959, 'f2_score': 0.8492095662748277, 'matthews_corrcoef': 0.4372018897922349, 'cohen_kappa': 0.4340398650555771, 'true_positive_rate': 0.8396793587174348, 'true_negative_rate': 0.6258992805755396, 'positive_predictive_value': 0.8895966029723992, 'negative_predictive_value': 0.5209580838323353, 'false_positive_rate': 0.37410071942446044, 'false_negative_rate': 0.16032064128256512, 'false_discovery_rate': 0.11040339702760085}
{'accuracy': 0.863013698630137, 'precision': 0.9272727272727272, 'recall': 0.8947368421052632, 'f1_score': 0.9107142857142856, 'f2_score': 0.9010600706713783, 'matthews_corrcoef': 0.618817104846334, 'cohen_kappa': 0.6169989506820567, 'true_positive_rate': 0.8947368421052632, 'true_negative_rate': 0.75, 'positive_predictive_value': 0.9272727272727272, 'negative_predictive_value': 0.66666666

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/Kunu/e/KUN-85
638/638 [==============================] - 2s 4ms/sample
{'accuracy': 0.822884012539185, 'precision': 0.9071729957805907, 'recall': 0.8617234468937875, 'f1_score': 0.8838643371017472, 'f2_score': 0.8704453441295547, 'matthews_corrcoef': 0.5149722795967155, 'cohen_kappa': 0.5119616577083983, 'true_positive_rate': 0.8617234468937875, 'true_negative_rate': 0.6834532374100719, 'positive_predictive_value': 0.9071729957805907, 'negative_predictive_value': 0.5792682926829268, 'false_positive_rate': 0.31654676258992803, 'false_negative_rate': 0.13827655310621242, 'false_discovery_rate': 0.09282700421940929}
{'accuracy': 0.863013698630137, 'precision': 0.9607843137254902, 'recall': 0.8596491228070176, 'f1_score': 0.9074074074074074, 'f2_score': 0.8781362007168458, 'matthews_corrcoef': 0.6623399563056614, 'cohen_kappa': 0.6473429951690821, 'true_positive_rate': 0.8596491228070176, 'true_negative_rate': 0.875, 'positive_predictive_value': 0.9607843

https://ui.neptune.ai/intelliscope/Kunu/e/KUN-86
638/638 [==============================] - 2s 4ms/sample
{'accuracy': 0.8150470219435737, 'precision': 0.8895705521472392, 'recall': 0.8717434869739479, 'f1_score': 0.8805668016194331, 'f2_score': 0.8752515090543259, 'matthews_corrcoef': 0.47150447265738965, 'cohen_kappa': 0.4710304801787496, 'true_positive_rate': 0.8717434869739479, 'true_negative_rate': 0.6115107913669064, 'positive_predictive_value': 0.8895705521472392, 'negative_predictive_value': 0.5704697986577181, 'false_positive_rate': 0.38848920863309355, 'false_negative_rate': 0.1282565130260521, 'false_discovery_rate': 0.11042944785276074}
{'accuracy': 0.8082191780821918, 'precision': 0.8909090909090909, 'recall': 0.8596491228070176, 'f1_score': 0.875, 'f2_score': 0.8657243816254415, 'matthews_corrcoef': 0.46516523867700615, 'cohen_kappa': 0.46379853095487933, 'true_positive_rate': 0.8596491228070176, 'true_negative_rate': 0.625, 'positive_predictive_value': 0.8909090909090909

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


...

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/Kunu/e/KUN-109
638/638 [==============================] - 2s 4ms/sample
{'accuracy': 0.8213166144200627, 'precision': 0.8857715430861723, 'recall': 0.8857715430861723, 'f1_score': 0.8857715430861723, 'f2_score': 0.8857715430861723, 'matthews_corrcoef': 0.47569960064012917, 'cohen_kappa': 0.47569960064012917, 'true_positive_rate': 0.8857715430861723, 'true_negative_rate': 0.5899280575539568, 'positive_predictive_value': 0.8857715430861723, 'negative_predictive_value': 0.5899280575539568, 'false_positive_rate': 0.41007194244604317, 'false_negative_rate': 0.11422845691382766, 'false_discovery_rate': 0.11422845691382766}
{'accuracy': 0.863013698630137, 'precision': 0.9122807017543859, 'recall': 0.9122807017543859, 'f1_score': 0.9122807017543859, 'f2_score': 0.9122807017543858, 'matthews_corrcoef': 0.5997807017543859, 'cohen_kappa': 0.5997807017543859, 'true_positive_rate': 0.9122807017543859, 'true_negative_rate': 0.6875, 'positive_predictive_value': 0.91

https://ui.neptune.ai/intelliscope/Kunu/e/KUN-110
638/638 [==============================] - 2s 4ms/sample
{'accuracy': 0.8495297805642633, 'precision': 0.8710865561694291, 'recall': 0.9478957915831663, 'f1_score': 0.9078694817658349, 'f2_score': 0.9314690823158723, 'matthews_corrcoef': 0.5151942033782873, 'cohen_kappa': 0.5015706124574796, 'true_positive_rate': 0.9478957915831663, 'true_negative_rate': 0.49640287769784175, 'positive_predictive_value': 0.8710865561694291, 'negative_predictive_value': 0.7263157894736842, 'false_positive_rate': 0.5035971223021583, 'false_negative_rate': 0.052104208416833664, 'false_discovery_rate': 0.1289134438305709}
{'accuracy': 0.8904109589041096, 'precision': 0.9152542372881356, 'recall': 0.9473684210526315, 'f1_score': 0.9310344827586206, 'f2_score': 0.9407665505226481, 'matthews_corrcoef': 0.6671002768745808, 'cohen_kappa': 0.6647531572904708, 'true_positive_rate': 0.9473684210526315, 'true_negative_rate': 0.6875, 'positive_predictive_value': 0.915

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/Kunu/e/KUN-111
